In [ ]:
"""
Switchee - Energy-on-Tap for the Informal City
A blockchain-secured, IoT-enabled, pre-paid micro-leasing platform
for solar-powered DC microgrids in low-income urban communities.

Installation Instructions for Google Colab:
1. Run: !pip install gradio pandas plotly
2. Copy this entire code
3. Run the cell
4. Click the Gradio public link to access the demo
"""

import gradio as gr
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import random
import json
import time

# ============================================================================
# SIMULATION DATA & STATE MANAGEMENT
# ============================================================================

class SwitcheeSimulator:
    def __init__(self):
        self.reset_system()

    def reset_system(self):
        """Initialize system state"""
        self.user_data = {
            "name": "Priya Sharma",
            "location": "Mumbai Basti",
            "phone": "+91 98765 43210",
            "plan": "Basic Light Plan",
            "balance": 150.0,
            "total_kwh_consumed": 45.2,
            "carbon_offset": 32.5,
            "savings": 1500.0,
            "connected_devices": 3,
            "subscription_date": "2024-10-15"
        }

        self.plans = {
            "Basic Light Plan": {
                "price": 300,
                "daily_limit": 2.5,
                "description": "Lights, fan, phone charging - 6 hours/day",
                "max_power": 100
            },
            "Productive Use Plan": {
                "price": 500,
                "daily_limit": 5.0,
                "description": "Basic + sewing machine or small tools - 10 hours/day",
                "max_power": 250
            },
            "Unlimited Family Plan": {
                "price": 800,
                "daily_limit": 12.0,
                "description": "Full household access - unlimited hours",
                "max_power": 500
            }
        }

        self.hub_status = {
            "solar_generation": 2.3,  # kW current
            "battery_level": 78,  # percentage
            "grid_health": "Optimal",
            "connected_homes": 45,
            "total_capacity": 15,  # kW
            "uptime": 99.7  # percentage
        }

        self.blockchain_transactions = []
        self.alerts = []

    def generate_usage_data(self, days=7):
        """Generate mock usage data"""
        dates = [(datetime.now() - timedelta(days=i)).strftime("%Y-%m-%d") for i in range(days)]
        usage = [random.uniform(1.5, 3.5) for _ in range(days)]
        return pd.DataFrame({"Date": dates[::-1], "Usage (kWh)": usage})

    def generate_solar_data(self, hours=24):
        """Generate solar generation pattern"""
        times = [f"{i:02d}:00" for i in range(hours)]
        # Realistic solar pattern: peaks at noon
        generation = [
            max(0, 2.5 * (1 - abs(i - 12) / 12) + random.uniform(-0.2, 0.2))
            for i in range(hours)
        ]
        return pd.DataFrame({"Time": times, "Generation (kW)": generation})

    def buy_credits(self, amount):
        """Simulate credit purchase"""
        self.user_data["balance"] += amount
        tx_hash = f"0x{random.randint(10**15, 10**16-1):016x}"
        tx = {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "type": "Credit Purchase",
            "amount": f"₹{amount}",
            "hash": tx_hash,
            "status": "Confirmed"
        }
        self.blockchain_transactions.insert(0, tx)
        return f"✅ Successfully purchased ₹{amount} credits!\n\nTransaction Hash: {tx_hash}\n\nNew Balance: ₹{self.user_data['balance']:.2f}"

    def change_plan(self, new_plan):
        """Change subscription plan"""
        old_plan = self.user_data["plan"]
        self.user_data["plan"] = new_plan
        tx_hash = f"0x{random.randint(10**15, 10**16-1):016x}"
        tx = {
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "type": "Plan Change",
            "amount": f"{old_plan} → {new_plan}",
            "hash": tx_hash,
            "status": "Confirmed"
        }
        self.blockchain_transactions.insert(0, tx)
        return f"✅ Plan changed successfully!\n\nFrom: {old_plan}\nTo: {new_plan}\n\nTransaction Hash: {tx_hash}"

    def trigger_alert(self, alert_type):
        """Generate system alerts"""
        alert_messages = {
            "Low Balance": f"⚠️ LOW BALANCE ALERT\n\nYour balance is ₹{self.user_data['balance']:.2f}. Top up to avoid service interruption.\n\nSMS sent to {self.user_data['phone']}",
            "High Solar": f"☀️ SUNSHINE FORECAST\n\nHigh solar generation expected today! Great time to use extra power.\n\nExpected: 3.5 kW peak generation",
            "Maintenance": f"🔧 MAINTENANCE SCHEDULED\n\nSwitchee Hub maintenance planned for tomorrow 10 AM - 12 PM.\n\nBattery backup will keep your power on."
        }
        alert = {
            "time": datetime.now().strftime("%H:%M:%S"),
            "type": alert_type,
            "message": alert_messages.get(alert_type, "System notification")
        }
        self.alerts.insert(0, alert)
        return alert_messages.get(alert_type, "Alert triggered")

# Initialize simulator
sim = SwitcheeSimulator()

# ============================================================================
# VISUALIZATION FUNCTIONS
# ============================================================================

def create_usage_chart():
    """Create usage over time chart"""
    df = sim.generate_usage_data(7)
    fig = px.line(df, x="Date", y="Usage (kWh)",
                  title="Your Weekly Energy Usage",
                  markers=True)
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict(size=12),
        height=300
    )
    fig.update_traces(line_color='#10b981', line_width=3)
    return fig

def create_solar_chart():
    """Create solar generation chart"""
    df = sim.generate_solar_data(24)
    fig = px.area(df, x="Time", y="Generation (kW)",
                  title="Today's Solar Generation Pattern")
    fig.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        font=dict(size=12),
        height=300
    )
    fig.update_traces(fillcolor='rgba(255,193,7,0.3)', line_color='#ffc107')
    return fig

def create_cost_comparison():
    """Create before/after cost comparison"""
    categories = ['Monthly Cost', 'Reliability', 'Safety Score', 'Carbon Impact']
    before = [800, 40, 30, 150]  # Before Switchee
    after = [300, 99, 95, -32]   # After Switchee (negative carbon = offset)

    fig = go.Figure(data=[
        go.Bar(name='Before Switchee', x=categories, y=before, marker_color='#ef4444'),
        go.Bar(name='After Switchee', x=categories, y=after, marker_color='#10b981')
    ])
    fig.update_layout(
        title="Impact Comparison: Before vs After Switchee",
        barmode='group',
        height=350,
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
    )
    return fig

# ============================================================================
# UI COMPONENT FUNCTIONS
# ============================================================================

def get_dashboard_html():
    """Generate main dashboard HTML"""
    user = sim.user_data
    plan = sim.plans[user["plan"]]
    hub = sim.hub_status

    return f"""
    <div style="font-family: Arial, sans-serif;">
        <div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 30px; border-radius: 10px; color: white; margin-bottom: 20px;">
            <h1 style="margin: 0; font-size: 2.5em;">⚡ Switchee Dashboard</h1>
            <p style="margin: 5px 0 0 0; font-size: 1.2em; opacity: 0.9;">Energy-on-Tap for Every Home</p>
        </div>

        <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(200px, 1fr)); gap: 15px; margin-bottom: 20px;">
            <div style="background: #10b981; padding: 20px; border-radius: 8px; color: white;">
                <div style="font-size: 0.9em; opacity: 0.9;">Current Balance</div>
                <div style="font-size: 2em; font-weight: bold;">₹{user['balance']:.2f}</div>
            </div>
            <div style="background: #3b82f6; padding: 20px; border-radius: 8px; color: white;">
                <div style="font-size: 0.9em; opacity: 0.9;">Your Plan</div>
                <div style="font-size: 1.3em; font-weight: bold;">{user['plan']}</div>
            </div>
            <div style="background: #f59e0b; padding: 20px; border-radius: 8px; color: white;">
                <div style="font-size: 0.9em; opacity: 0.9;">Total Savings</div>
                <div style="font-size: 2em; font-weight: bold;">₹{user['savings']:.0f}</div>
            </div>
            <div style="background: #22c55e; padding: 20px; border-radius: 8px; color: white;">
                <div style="font-size: 0.9em; opacity: 0.9;">Carbon Offset</div>
                <div style="font-size: 2em; font-weight: bold;">{user['carbon_offset']} kg</div>
            </div>
        </div>

        <div style="background: white; padding: 20px; border-radius: 8px; border: 1px solid #e5e7eb; margin-bottom: 20px;">
            <h3 style="margin-top: 0;">👤 User Profile</h3>
            <p><strong>Name:</strong> {user['name']}</p>
            <p><strong>Location:</strong> {user['location']}</p>
            <p><strong>Phone:</strong> {user['phone']}</p>
            <p><strong>Subscription Since:</strong> {user['subscription_date']}</p>
            <p><strong>Connected Devices:</strong> {user['connected_devices']}</p>
            <p><strong>Total Energy Consumed:</strong> {user['total_kwh_consumed']} kWh</p>
        </div>

        <div style="background: white; padding: 20px; border-radius: 8px; border: 1px solid #e5e7eb;">
            <h3 style="margin-top: 0;">🏢 Switchee Hub Status</h3>
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 10px;">
                <div>
                    <p><strong>Solar Generation:</strong> {hub['solar_generation']} kW</p>
                    <p><strong>Battery Level:</strong> {hub['battery_level']}%</p>
                    <p><strong>Grid Health:</strong> <span style="color: #10b981;">{hub['grid_health']}</span></p>
                </div>
                <div>
                    <p><strong>Connected Homes:</strong> {hub['connected_homes']}</p>
                    <p><strong>Total Capacity:</strong> {hub['total_capacity']} kW</p>
                    <p><strong>System Uptime:</strong> {hub['uptime']}%</p>
                </div>
            </div>
        </div>
    </div>
    """

def get_plan_info(plan_name):
    """Get plan details"""
    plan = sim.plans[plan_name]
    return f"""
    ### {plan_name}

    **Monthly Price:** ₹{plan['price']}

    **Daily Limit:** {plan['daily_limit']} kWh

    **Max Power Draw:** {plan['max_power']} W

    **Description:** {plan['description']}

    ---

    **What's Included:**
    - 24/7 Reliable Power
    - Real-time Monitoring
    - SMS Alerts
    - Blockchain-secured Transactions
    - Carbon-negative Energy
    """

def refresh_dashboard():
    """Refresh all dashboard components"""
    return (
        get_dashboard_html(),
        create_usage_chart(),
        create_solar_chart(),
        get_blockchain_table(),
        get_alerts_html()
    )

def get_blockchain_table():
    """Generate blockchain transactions table"""
    if not sim.blockchain_transactions:
        return pd.DataFrame({"Message": ["No transactions yet"]})
    return pd.DataFrame(sim.blockchain_transactions[:10])

def get_alerts_html():
    """Generate alerts HTML"""
    if not sim.alerts:
        return "<p style='color: #6b7280;'>No recent alerts</p>"

    html = ""
    for alert in sim.alerts[:5]:
        html += f"""
        <div style="background: #fef3c7; padding: 15px; border-radius: 8px; margin-bottom: 10px; border-left: 4px solid #f59e0b;">
            <div style="font-weight: bold; color: #92400e;">{alert['time']} - {alert['type']}</div>
            <div style="color: #78350f; margin-top: 5px;">{alert['message']}</div>
        </div>
        """
    return html

# ============================================================================
# GRADIO INTERFACE
# ============================================================================

with gr.Blocks(title="Switchee - Energy-on-Tap Platform", theme=gr.themes.Soft()) as demo:

    gr.Markdown("""
    # ⚡ Switchee - Energy-on-Tap for the Informal City
    ### Pay for Power, Not the Panel. Reliable, Safe, and Affordable Electricity for Every Home.
    """)

    with gr.Tabs():

        # ===== TAB 1: DASHBOARD =====
        with gr.Tab("📊 Dashboard"):
            dashboard_html = gr.HTML(get_dashboard_html())

            with gr.Row():
                with gr.Column():
                    usage_chart = gr.Plot(create_usage_chart(), label="Energy Usage Trend")
                with gr.Column():
                    solar_chart = gr.Plot(create_solar_chart(), label="Solar Generation")

            comparison_chart = gr.Plot(create_cost_comparison(), label="Impact Analysis")

            refresh_btn = gr.Button("🔄 Refresh Dashboard", variant="primary")

        # ===== TAB 2: BUY CREDITS =====
        with gr.Tab("💳 Buy Credits"):
            gr.Markdown("## Top Up Your Energy Credits")
            gr.Markdown("Pay via UPI and get instant access to energy. All transactions are secured on blockchain.")

            with gr.Row():
                with gr.Column():
                    credit_amount = gr.Slider(
                        minimum=50, maximum=1000, step=50, value=200,
                        label="Credit Amount (₹)"
                    )
                    upi_id = gr.Textbox(
                        value="priya@oksbi",
                        label="UPI ID",
                        placeholder="yourname@upi"
                    )
                    buy_btn = gr.Button("💰 Buy Credits", variant="primary", size="lg")

                with gr.Column():
                    purchase_result = gr.Textbox(
                        label="Transaction Result",
                        lines=6,
                        interactive=False
                    )

            gr.Markdown("### Quick Top-Up Options")
            with gr.Row():
                quick_50 = gr.Button("₹50")
                quick_100 = gr.Button("₹100")
                quick_200 = gr.Button("₹200")
                quick_500 = gr.Button("₹500")

        # ===== TAB 3: MANAGE PLAN =====
        with gr.Tab("📋 Manage Plan"):
            gr.Markdown("## Change Your Subscription Plan")

            with gr.Row():
                with gr.Column():
                    plan_selector = gr.Radio(
                        choices=list(sim.plans.keys()),
                        value=sim.user_data["plan"],
                        label="Select Plan"
                    )
                    change_plan_btn = gr.Button("✅ Change Plan", variant="primary")

                with gr.Column():
                    plan_details = gr.Markdown(get_plan_info(sim.user_data["plan"]))
                    plan_change_result = gr.Textbox(
                        label="Result",
                        lines=4,
                        interactive=False
                    )

        # ===== TAB 4: ALERTS & NOTIFICATIONS =====
        with gr.Tab("🔔 Alerts"):
            gr.Markdown("## System Alerts & Notifications")

            alerts_html = gr.HTML(get_alerts_html())

            gr.Markdown("### Trigger Test Alerts")
            with gr.Row():
                alert_low = gr.Button("⚠️ Low Balance Alert")
                alert_solar = gr.Button("☀️ High Solar Alert")
                alert_maint = gr.Button("🔧 Maintenance Alert")

            alert_result = gr.Textbox(label="Alert Message", lines=4, interactive=False)

        # ===== TAB 5: BLOCKCHAIN LEDGER =====
        with gr.Tab("🔗 Blockchain Ledger"):
            gr.Markdown("## Transaction History")
            gr.Markdown("All transactions are recorded on Hedera Hashgraph for transparency and security.")

            blockchain_table = gr.Dataframe(
                get_blockchain_table(),
                label="Recent Transactions"
            )
            refresh_blockchain = gr.Button("🔄 Refresh Ledger")

        # ===== TAB 6: IOT DEVICES =====
        with gr.Tab("🔌 IoT Devices"):
            gr.Markdown("## Connected Smart Plugs")

            gr.Markdown("""
            ### Device Status Monitor

            | Device | Status | Power Draw | Location |
            |--------|--------|------------|----------|
            | Smart Plug #1 | 🟢 Active | 45 W | Living Room |
            | Smart Plug #2 | 🟢 Active | 15 W | Bedroom |
            | Smart Plug #3 | 🔴 Inactive | 0 W | Kitchen |

            ---

            ### Real-Time Power Control Demo

            Simulate cutting power when balance reaches zero:
            """)

            with gr.Row():
                with gr.Column():
                    device_status = gr.Textbox(
                        value="💡 Device is ON - Drawing 45W",
                        label="Device Status",
                        interactive=False
                    )
                    simulate_zero = gr.Button("⚠️ Simulate Zero Balance", variant="stop")
                    restore_power = gr.Button("✅ Top Up & Restore Power", variant="primary")

                with gr.Column():
                    power_log = gr.Textbox(
                        label="Power Control Log",
                        lines=8,
                        interactive=False
                    )

        # ===== TAB 7: ABOUT =====
        with gr.Tab("ℹ️ About Switchee"):
            gr.Markdown("""
            ## The Switchee Solution

            ### 🎯 The Problem We Solve

            Millions in urban India live in informal housing with:
            - ❌ Unreliable, expensive electricity (₹800+/month)
            - ❌ Dangerous illegal connections
            - ❌ 3-5x higher costs than grid tariff
            - ❌ Complete exclusion from clean energy

            ### ✅ Our Solution

            **Switchee** is a blockchain-secured, IoT-enabled, pre-paid micro-leasing platform for solar-powered DC microgrids.

            #### How It Works:

            1. **Community Solar Hub**: We install a centralized solar + battery system
            2. **Smart Plugs**: Each home gets an IoT-enabled meter and circuit breaker
            3. **Pre-Paid Credits**: Buy energy credits via UPI/USSD - No bills, no defaults
            4. **Blockchain Security**: All transactions recorded on Hedera Hashgraph
            5. **Tiered Plans**: Choose from Basic, Productive, or Unlimited plans

            ### 📊 Impact

            - 💰 **62.5% Cost Reduction**: ₹800/month → ₹300/month
            - 🔋 **99.7% Uptime**: Reliable 24/7 power
            - 🌱 **Carbon Negative**: Clean solar energy
            - 🛡️ **Safe**: No more illegal connections

            ### 🏆 Technology Stack

            - **Frontend**: Gradio UI + USSD for feature phones
            - **IoT**: ESP32-based Smart Plugs with real-time monitoring
            - **Blockchain**: Hedera Hashgraph for transaction ledger
            - **Payments**: UPI integration for instant top-ups
            - **Backend**: Microservices architecture for scalability

            ### 🌍 Vision

            Making clean, reliable energy accessible to **every home**, not just the privileged.

            ---

            **Developed for the Energy-as-a-Service Hackathon**

            *"Pay for Power, Not the Panel"*
            """)

    # ============================================================================
    # EVENT HANDLERS
    # ============================================================================

    # Dashboard refresh
    refresh_btn.click(
        refresh_dashboard,
        outputs=[dashboard_html, usage_chart, solar_chart, blockchain_table, alerts_html]
    )

    # Buy credits
    def handle_buy(amount):
        result = sim.buy_credits(amount)
        return result, get_dashboard_html(), get_blockchain_table()

    buy_btn.click(
        handle_buy,
        inputs=[credit_amount],
        outputs=[purchase_result, dashboard_html, blockchain_table]
    )

    # Quick top-up buttons
    quick_50.click(lambda: sim.buy_credits(50), outputs=[purchase_result, dashboard_html, blockchain_table])
    quick_100.click(lambda: sim.buy_credits(100), outputs=[purchase_result, dashboard_html, blockchain_table])
    quick_200.click(lambda: sim.buy_credits(200), outputs=[purchase_result, dashboard_html, blockchain_table])
    quick_500.click(lambda: sim.buy_credits(500), outputs=[purchase_result, dashboard_html, blockchain_table])

    # Change plan
    plan_selector.change(
        lambda p: get_plan_info(p),
        inputs=[plan_selector],
        outputs=[plan_details]
    )

    change_plan_btn.click(
        lambda p: (sim.change_plan(p), get_dashboard_html(), get_blockchain_table()),
        inputs=[plan_selector],
        outputs=[plan_change_result, dashboard_html, blockchain_table]
    )

    # Alerts
    alert_low.click(lambda: (sim.trigger_alert("Low Balance"), get_alerts_html()),
                    outputs=[alert_result, alerts_html])
    alert_solar.click(lambda: (sim.trigger_alert("High Solar"), get_alerts_html()),
                      outputs=[alert_result, alerts_html])
    alert_maint.click(lambda: (sim.trigger_alert("Maintenance"), get_alerts_html()),
                      outputs=[alert_result, alerts_html])

    # Blockchain refresh
    refresh_blockchain.click(get_blockchain_table, outputs=[blockchain_table])

    # IoT device simulation
    def simulate_power_cut():
        log = f"""
[{datetime.now().strftime('%H:%M:%S')}] ⚠️ Balance reached ₹0.00
[{datetime.now().strftime('%H:%M:%S')}] 🔴 Smart Plug #1 - Power CUT
[{datetime.now().strftime('%H:%M:%S')}] 📱 SMS Alert sent to user
[{datetime.now().strftime('%H:%M:%S')}] 🔗 Transaction recorded on blockchain
        """
        return "🔴 Device is OFF - Zero Balance!", log

    def restore_power_sim():
        sim.buy_credits(100)
        log = f"""
[{datetime.now().strftime('%H:%M:%S')}] 💳 ₹100 credits purchased
[{datetime.now().strftime('%H:%M:%S')}] ✅ Balance updated: ₹{sim.user_data['balance']:.2f}
[{datetime.now().strftime('%H:%M:%S')}] 🟢 Smart Plug #1 - Power RESTORED
[{datetime.now().strftime('%H:%M:%S')}] 💡 Device drawing 45W
[{datetime.now().strftime('%H:%M:%S')}] 🔗 Transaction recorded on blockchain
        """
        return "💡 Device is ON - Drawing 45W", log

    simulate_zero.click(simulate_power_cut, outputs=[device_status, power_log])
    restore_power.click(restore_power_sim, outputs=[device_status, power_log])

# ============================================================================
# LAUNCH
# ============================================================================

if __name__ == "__main__":
    demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://31606ab9a699c66496.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
